# FINN - How to work with ONNX

<font size="3">This notebook should give an overview of ONNX ProtoBuf, help to create and manipulate an ONNX model and use FINN functions to work with it. There may be overlaps to other notebooks, like [FINN-ModelWrapper](FINN-ModelWrapper.ipynb) and [FINN-CustomOps](FINN-CustomOps.ipynb), but this notebook will give an overview about the handling of ONNX models in FINN. </font>

## Outline
* #### How to create a simple model
* #### How to manipulate an ONNX model

### How to create a simple model

<font size="3">To explain how to create an ONNX graph a simple example with mathematical operations is used. All nodes are from the [standard operations library of ONNX](https://github.com/onnx/onnx/blob/master/docs/Operators.md).

First ONNX is imported, then the helper function can be used to make a node.</font>

In [1]:
import onnx

Add1_node = onnx.helper.make_node(
    'Add',
    inputs=['in1', 'in2'],
    outputs=['sum1'],
    name='Add1'
)

<font size="3">The first attribute of the node is the operation type. In this case it is `'Add'`, so it is an adder node. Then the input names are passed to the node and at the end a name is assigned to the output.
    
For this example we want two other adder nodes, one abs node and the output shall be rounded so one round node is needed. </font>

In [2]:
Add2_node = onnx.helper.make_node(
    'Add',
    inputs=['sum1', 'in3'],
    outputs=['sum2'],
    name='Add2',
)

Add3_node = onnx.helper.make_node(
    'Add',
    inputs=['abs1', 'abs1'],
    outputs=['sum3'],
    name='Add3',
)

Abs_node = onnx.helper.make_node(
    'Abs',
    inputs=['sum2'],
    outputs=['abs1'],
    name='Abs'
)

Round_node = onnx.helper.make_node(
    'Round',
    inputs=['sum3'],
    outputs=['out1'],
    name='Round',
)


<font size="3">The names of the inputs and outputs of the nodes give already an idea of the structure of the resulting graph. In order to integrate the nodes into a graph environment, the inputs and outputs of the graph have to be specified first. In ONNX all data edges are processed as tensors. So with the helper function tensor value infos are created for the input and output tensors of the graph. Float from ONNX is used as data type. </font>

In [3]:
in1 = onnx.helper.make_tensor_value_info("in1", onnx.TensorProto.FLOAT, [4, 4])
in2 = onnx.helper.make_tensor_value_info("in2", onnx.TensorProto.FLOAT, [4, 4])
in3 = onnx.helper.make_tensor_value_info("in3", onnx.TensorProto.FLOAT, [4, 4])
out1 = onnx.helper.make_tensor_value_info("out1", onnx.TensorProto.FLOAT, [4, 4])

<font size="3">Now the graph can be built. First all nodes are passed. Here it is to be noted that it requires a certain sequence. The nodes must be instantiated in their dependencies to each other. This means Add2 must not be listed before Add1, because Add2 depends on the result of Add1. A name is then assigned to the graph. This is followed by the inputs and outputs. 

`value_info` of the graph contains the remaining tensors within the graph. When creating the nodes we have already defined names for the inner data edges and now these are assigned tensors of the datatype float and a certain shape.</font>

In [4]:
 graph = onnx.helper.make_graph(
        nodes=[
            Add1_node,
            Add2_node,
            Abs_node,
            Add3_node,
            Round_node,
        ],
        name="simple_graph",
        inputs=[in1, in2, in3],
        outputs=[out1],
        value_info=[
            onnx.helper.make_tensor_value_info("sum1", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("sum2", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("abs1", onnx.TensorProto.FLOAT, [4, 4]),
            onnx.helper.make_tensor_value_info("sum3", onnx.TensorProto.FLOAT, [4, 4]),
        ],
    )


<font size="3">**Important**: In our example, the shape of the tensors does not change during the calculation. This is not always the case. So you have to make sure that you specify the shape correctly.

Now a model can be created from the graph and saved using the `.save` function. The model is saved in .onnx format and can be reloaded with `onnx.load()`. This also means that you can easily share your own model in .onnx format with others.</font>

In [5]:
onnx_model = onnx.helper.make_model(graph, producer_name="simple-model")
onnx.save(onnx_model, 'simple_model.onnx')

<font size='3'>To visualize the created model, [netron](https://github.com/lutzroeder/netron) can be used. Netron is a visualizer for neural network, deep learning and machine learning models. <font>

In [6]:
import netron
netron.start('simple_model.onnx', port=8081, host="0.0.0.0")

Serving 'simple_model.onnx' at http://0.0.0.0:8081


In [7]:
%%html
<iframe src="http://0.0.0.0:8081/" style="position: relative; width: 100%;" height="400"></iframe>

<font size="3">Netron also allows you to interactively explore the model. If you click on a node, the node attributes will be displayed. 

In order to test the resulting model, a function is first written in Python that calculates the expected output. Because numpy arrays are to be used, numpy is imported first.</font>

In [8]:
import numpy as np

def expected_output(in1, in2, in3):
    sum1 = np.add(in1, in2)
    sum2 = np.add(sum1, in3)
    abs1 = np.absolute(sum2)
    sum3 = np.add(abs1, abs1)
    return np.round(sum3)

<font size="3">Then the values for the three inputs are calculated. Random numbers are used.</font>

In [9]:
in1_values =np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in2_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)
in3_values = np.asarray(np.random.uniform(low=-5, high=5, size=(4,4)), dtype=np.float32)

<font size="3">We can easily pass the values to the function we just wrote to calculate the expected result. For the created model the inputs must be summarized in a dictionary, which is then passed on to the model.</font>

In [10]:
input_dict = {}
input_dict["in1"] = in1_values
input_dict["in2"] = in2_values
input_dict["in3"] = in3_values

<font size="3">To run the model and calculate the output, [onnxruntime](https://github.com/microsoft/onnxruntime) can be used. ONNX Runtime is a performance-focused complete scoring engine for Open Neural Network Exchange (ONNX) models from Microsoft. The `.InferenceSession` function is used to create a session of the model and `.run` is used to execute the model. </font>

In [11]:
import onnxruntime as rt

sess = rt.InferenceSession(onnx_model.SerializeToString())
output = sess.run(None, input_dict)

<font size="3">The input values are also transferred to the reference function. Now the output of the execution of the model can be compared with that of the reference. </font>

In [12]:
ref_output= expected_output(in1_values, in2_values, in3_values)
print("The output of the ONNX model is: \n{}".format(output[0]))
print("\nThe output of the reference function is: \n{}".format(ref_output))

if (output[0] == ref_output).all():
    print("\nThe results are the same!")
else:
    raise Exception("Something went wrong, the output of the model doesn't match the expected output!")

The output of the ONNX model is: 
[[ 8.  3. 10.  8.]
 [ 4. 22.  5. 11.]
 [ 7. 15.  2.  8.]
 [ 2. 12.  8.  3.]]

The output of the reference function is: 
[[ 8.  3. 10.  8.]
 [ 4. 22.  5. 11.]
 [ 7. 15.  2.  8.]
 [ 2. 12.  8.  3.]]

The results are the same!


<font size="3">Now that we have verified that the model works as we expected it to, we can continue working with the graph. </font>

### How to manipulate an ONNX model

<font size="3">In the model there are two successive adder nodes. An adder node in ONNX can only add two inputs, but there is also the [**sum**](https://github.com/onnx/onnx/blob/master/docs/Operators.md#Sum) node, which can process more than one input. So it would be a reasonable change of the graph to combine the two successive adder nodes to one sum node. </font>

<font size="3">In the following we assume that we do not know the appearance of the model, so we first try to identify whether there are two consecutive adders in the graph and then convert them into a sum node. 

Here we make use of FINN. FINN provides a thin wrapper around the model which provides several additional helper functions to manipulate the graph. The code can be found [here](https://github.com/Xilinx/finn/blob/dev/src/finn/core/modelwrapper.py) and you can find a more detailed description in the notebook [FINN-ModelWrapper](FINN-ModelWrapper.ipynb).</font>

In [13]:
from finn.core.modelwrapper import ModelWrapper
finn_model = ModelWrapper(onnx_model)

def get_node_id(model):
    node_index = {}
    node_ind = 0
    for node in model.graph.node:
        node_index[node_ind] = node.name
        node_ind += 1

<font size="3">Now we design a function that searches for adder nodes in the graph and returns the found nodes. </font>

In [14]:
def identify_adder_nodes(model):
    add_nodes = []
    for node in model.graph.node:
        if node.op_type == "Add":
            add_nodes.append(node)
    return add_nodes
            

<font size="3">The function iterates over all nodes of the model and if the operation type is `"Add"` the node will be stored in `add_nodes`. At the end `add_nodes` is returned.

If we apply this to our model, three nodes should be returned.</font> 

In [15]:
add_nodes = identify_adder_nodes(finn_model)
for node in add_nodes:
    print("Found adder node: {}".format(node.name))

Found adder node: Add1
Found adder node: Add2
Found adder node: Add3


<font size="3">Among other helper functions, ModelWrapper offers two functions to determine the preceding and succeeding node of a node. However, these functions are not getting a node as input, but can determine the consumer or producer of a tensor. We write two functions that uses these helper functions to determine the previous and the next node of a node.</font>

In [16]:
def find_predecessor(model, node):
    predecessors = []
    for i in range(len(node.input)):
        producer = model.find_producer(node.input[i])
        predecessors.append(producer)
    return predecessors
        

def find_successor(model, node):
    successors = []
    for i in range(len(node.output)):
        consumer = model.find_consumer(node.output[i])
        successors.append(consumer)
    return successors

<font size="3">The first function uses `find_producer` from ModelWrapper to create a list of the producers of the inputs of the given node. So the returned list is indirectly filled with the predecessors of the node. The second function works in a similar way, `find_consumer` from ModelWrapper is used to find the consumers of the output tensors of the node and so a list with the successors can be created. 

So now we can find out which adder node has an adder node as successor</font>

In [25]:
def adder_pair(model, node):
    adder_pairs = []
    node_pair = []
    successor_list = find_successor(model, node)
    
    for successor in successor_list:
        if successor.op_type == "Add":
            node_pair.append(node)
            node_pair.append(successor)
            adder_pairs.append((node_pair))
            node_pair = []
    return adder_pairs
            

The function gets a node and the model as input. Two empty lists are created to be filled with a list of adder node pairs that can be returned as result of the function. Then the function `find_successor` is used to return all of the successors of the node. If one of the successors is an adder node, the node is saved in `node_pair` together with the successive adder node and put in the list `adder_pairs`. Then the temporary list is cleaned and can be filled with the next adder node pair.

In [27]:
for node in add_nodes:
    add_pairs = adder_pair(finn_model, node)
    if len(add_pairs) != 0:
        substitute_pair = add_pairs
        print("Found following pair that could be replaced by a sum node:")
        for node_pair in add_pairs:
            for node in node_pair:
                print(node.name)

Found following pair that could be replaced by a sum node:
Add1
Add2


<font size="3">Now that the pair to be replaced has been identified, a sum node can be instantiated and inserted into the graph at the correct position. 

First of all, the inputs must be determined. For this the adder nodes inputs are used minus the input, which corresponds to the output of the other adder node.</font>

In [74]:
input_list = []
for i in range(len(substitute_pair)):
    if i == 0:
        for j in range(len(substitute_pair[i].input)):
            if substitute_pair[i].input[j] != substitute_pair[i+1].output[0]:
                input_list.append(substitute_pair[i].input[j])
    else:
        for j in range(len(substitute_pair[i].input)):
            if substitute_pair[i].input[j] != substitute_pair[i-1].output[0]:
                input_list.append(substitute_pair[i].input[j])
        

['in1', 'in2', 'in3']